# 阶段一 语音识别

## 第一步：安装依赖环境

In [ ]:
# 安装 stable-ts 和 ffmpeg
!pip install -U stable-ts
!sudo apt update && sudo apt install ffmpeg

## 第二步：准备音频/视频文件

In [ ]:
import os
from google.colab import files, drive

# @markdown 请选择文件来源，然后点击左侧运行按钮 ▶️。

source_type = "挂载 Google Drive" # @param ["直接上传本地文件", "挂载 Google Drive", "使用示例音频"]
drive_file_path = "/content/drive/MyDrive/audios/audio.mp3" # @param {type:"string"}
# @markdown > *注：如果选择 Google Drive，请先运行一次挂载，去左侧文件栏复制路径填入上方，或留空在运行时根据提示输入。*

# 初始化全局路径变量
GLOBAL_FILE_PATH = ""

def prepare_file():
    global GLOBAL_FILE_PATH

    if source_type == "直接上传本地文件":
        print("⬆️ 正在等待文件上传，请点击下方的【选择文件】按钮...")
        uploaded = files.upload()
        if not uploaded:
            print("❌ 未上传任何文件。")
            return
        # 自动获取上传的文件名（处理字典键值）
        filename = next(iter(uploaded))
        GLOBAL_FILE_PATH = os.path.abspath(filename)
        print(f"✅ 本地文件已接收: {filename}")

    elif source_type == "挂载 Google Drive":
        print("🔄 正在挂载 Google Drive...")
        drive.mount('/content/drive')

        target_path = drive_file_path.strip()
        # 如果表单没填，弹出交互输入框
        if not target_path:
            print("\n⚠️ 检测到路径为空。请在左侧文件栏找到您的文件 -> 右键 'Copy path' -> 粘贴到下方输入框：")
            target_path = input("👉 请粘贴文件路径并回车: ").strip()

        # 去除可能存在的引号
        target_path = target_path.replace('"', '').replace("'", "")

        if os.path.exists(target_path):
            GLOBAL_FILE_PATH = target_path
            print(f"✅ Drive 文件路径已确认: {GLOBAL_FILE_PATH}")
        else:
            print(f"❌ 找不到文件: {target_path}。请检查路径后重新运行。")

    elif source_type == "使用示例音频":
        url = "https://upload.wikimedia.org/wikipedia/commons/e/e4/JFK_we_choose_to_go_to_the_moon_speech.ogg"
        save_name = "test_audio.ogg"
        print("⬇️ 正在下载示例音频...")
        os.system(f"wget {url} -O {save_name}")
        GLOBAL_FILE_PATH = os.path.abspath(save_name)
        print(f"✅ 示例文件已就绪")

    # 最终结果汇总
    print("-" * 30)
    if GLOBAL_FILE_PATH and os.path.exists(GLOBAL_FILE_PATH):
        print(f"🎉 文件准备成功！\n📂 路径已自动锁定: {GLOBAL_FILE_PATH}")
    else:
        print("❌ 文件未就绪，请重新检查并运行此步骤。")

# 执行主逻辑
prepare_file()

### 可选：音乐人声分离预处理

如果音频包含背景音乐（如歌曲），建议运行此步骤提取纯人声。

In [ ]:
print("正在安装分离工具")
!pip install -q demucs torchcodec

In [ ]:
enable_separation = True # @param {type:"boolean"}

if enable_separation:
    if 'GLOBAL_FILE_PATH' not in globals() or not os.path.exists(GLOBAL_FILE_PATH):
        print("未发现全局文件路径")
    else:
        import shutil
        import subprocess
        import os

        original_ext = os.path.splitext(GLOBAL_FILE_PATH)[1]
        temp_input = "/content/temp_vocal_input" + original_ext
        shutil.copy(GLOBAL_FILE_PATH, temp_input)

        print(f"正在提取人声与伴奏")

        try:
            subprocess.run(
                ["python3", "-m", "demucs.separate", "-n", "mdx_extra", "--two-stems=vocals", "--out", "/content/separated", temp_input],
                check=True
            )

            base_output_path = "/content/separated/mdx_extra/temp_vocal_input"
            vocal_path = os.path.join(base_output_path, "vocals.wav")
            instr_path = os.path.join(base_output_path, "no_vocals.wav")
            base_filename = os.path.splitext(os.path.basename(GLOBAL_FILE_PATH))[0]

            if os.path.exists(vocal_path):
                final_vocal_name = f"/content/{base_filename}_vocals.wav"
                shutil.move(vocal_path, final_vocal_name)
                GLOBAL_FILE_PATH = final_vocal_name
                print(f"人声提取成功: {final_vocal_name}")

            if os.path.exists(instr_path):
                final_instr_name = f"/content/{base_filename}_instr.wav"
                shutil.move(instr_path, final_instr_name)
                print(f"伴奏提取成功: {final_instr_name}")

        except Exception as e:
            print(f"运行异常: {e}")
        finally:
            if os.path.exists(temp_input):
                os.remove(temp_input)
            if os.path.exists("/content/separated"):
                shutil.rmtree("/content/separated")
else:
    print("跳过人声分离")

## 第三步：配置参数并执行识别

功能说明：
- **自动填充**：代码会优先读取上一步生成的 GLOBAL_FILE_PATH。
- **自定义覆盖**：如果你想临时换个文件，也可以在 manual_file_path 中手动填入新路径覆盖自动路径。

In [ ]:
# @title 📝 可选：参考文本（软引导） { display-mode: "form" }
# @markdown 如果你有现成的歌词或文案，粘贴在下面可以显著降低识别错误率（Initial Prompt 机制）。
reference_text = "" # @param {type:"string"}

INITIAL_PROMPT = reference_text.strip()

if INITIAL_PROMPT:
    print(f"✅ 已记录参考文本 (前50字): {INITIAL_PROMPT[:50]}...")
else:
    print("ℹ️ 未提供参考文本，模型将进行自主识别。")

In [ ]:
import stable_whisper
import os

model_size = "medium" # @param ["tiny", "base", "small", "medium", "large-v3"]
language = "ja" # @param ["Auto", "zh", "en", "ja", "ko", "fr", "de", "es"]
manual_file_path = "" # @param {type:"string"}

# --- 路径处理逻辑 ---
target_file = ""
if manual_file_path.strip():
    target_file = manual_file_path.strip()
elif 'GLOBAL_FILE_PATH' in globals() and GLOBAL_FILE_PATH:
    target_file = GLOBAL_FILE_PATH
else:
    if os.path.exists("/content/test_audio.ogg"):
        target_file = "/content/test_audio.ogg"
    else:
        raise FileNotFoundError("❌ 未找到文件，请先运行步骤二。")

# --- 加载模型 ---
print("-" * 30)
print(f"🚀 正在加载模型 [{model_size}] ...")
model = stable_whisper.load_model(model_size)

# --- 准备参数 ---
transcribe_args = {
    "vad": True  # 开启语音活动检测，对音乐场景非常重要
}

if language != "Auto":
    transcribe_args["language"] = language

# 注入参考文本作为 initial_prompt
if 'INITIAL_PROMPT' in globals() and INITIAL_PROMPT:
    transcribe_args["initial_prompt"] = INITIAL_PROMPT

# --- 执行识别 ---
print(f"🎙️ 开始识别: {os.path.basename(target_file)}")
print("⏳ 正在计算逐词时间轴，请稍候...")

result = model.transcribe(target_file, **transcribe_args)

print("✅ 识别完成！")

## 第四步：导出结果

In [ ]:
import os
import json
from datetime import datetime

base_name = os.path.splitext(os.path.basename(target_file))[0]
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_prefix = f"{base_name}_{timestamp}"

print(f"正在导出文件，前缀为: {output_prefix}")

ass_filename = f"{output_prefix}_karaoke.ass"
result.to_ass(ass_filename)
print(f" -> 已生成: {ass_filename}")

json_full_filename = f"{output_prefix}_data_full.json"
result.save_as_json(json_full_filename)
print(f" -> 已生成: {json_full_filename}")

raw_data = result.to_dict()
cleaned_segments = []

for index, seg in enumerate(raw_data.get("segments", [])):
    new_seg = {
        "id": seg.get("id", index),
        "start": round(seg.get("start", 0), 3),
        "end": round(seg.get("end", 0), 3),
        "text": seg.get("text", "").strip(),
        "words": []
    }

    if "words" in seg:
        for word in seg["words"]:
            new_word = {
                "word": word.get("word", ""),
                "start": round(word.get("start", 0), 3),
                "end": round(word.get("end", 0), 3)
            }
            new_seg["words"].append(new_word)

    cleaned_segments.append(new_seg)

cleaned_data = {
    "language": raw_data.get("language", "auto"),
    "segments": cleaned_segments
}

json_clean_filename = f"{output_prefix}_data.json"
with open(json_clean_filename, 'w', encoding='utf-8') as f:
    json.dump(cleaned_data, f, indent=2, ensure_ascii=False)
print(f" -> 已生成: {json_clean_filename}")

srt_filename = f"{output_prefix}_words.srt"
result.to_srt_vtt(srt_filename, word_level=True)
print(f" -> 已生成: {srt_filename}")

print("-" * 30)
print("所有文件导出完毕！")


## 第五步：批量下载结果

In [ ]:
import zipfile
from google.colab import files
import os

zip_filename = f"{output_prefix}_results.zip"

try:
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for f in [ass_filename, json_full_filename, json_clean_filename, srt_filename]:
            if os.path.exists(f):
                zipf.write(f)

    print(f"已打包: {zip_filename}")
    files.download(zip_filename)
except Exception as e:
    print(f"打包下载出错: {e}")
    print("请在左侧文件夹手动下载。")

# 阶段二 翻译与智能对齐

使用以下提示词让AI生成处理后的文件：

### 字幕对齐与翻译专家

```
# Role
你是一个资深的字幕对齐与翻译专家。你的核心任务是处理语音识别（ASR）生成的源语言数据，将其翻译为中文，并基于“语义对应关系”将源语言的时间戳精确迁移到中文字幕上。

# Workflow
1. **翻译**: 将输入的 `text` 翻译成自然、流畅、符合语境的简体中文。
2. **对齐 (Mapping)**: 分析源单词（Source Words）与翻译后的中文词（Target Words）之间的语义关系，并分配 `start` 和 `end` 时间戳。

# Rules for Time Alignment (核心规则)
请严格遵循以下特殊情况的处理逻辑：

1. **基本对应 (One-to-One)**:
   - 如果英文词 "World" (1.0s-1.5s) 对应中文词 "世界"，则 "世界" 的时间戳也是 (1.0s-1.5s)。

2. **语序颠倒/倒装 (Inverted Word Order)**:
   - **挑战**: 英文 "The book [t1] that I read [t2]" -> 中文 "我读的 [t2] 那本书 [t1]"。
   - **规则**: 请优先**遵循语音的时间**。即：当听到 "The book" 时，中文字幕中的 "那本书" 应该亮起（即使它在句尾）。
   - **例外**: 如果倒装导致视觉跳跃过于混乱，请将整个短语视为一个整体，取该短语在源语音中的总时间范围，平均分配给中文短语。

3. **多对一 (Many-to-One)**:
   - **情况**: 英文 "Ladies and gentlemen" (3个词, 总长2秒) -> 中文 "各位" (1个词)。
   - **规则**: 将源语言这3个词的时间范围合并 (Start of first ~ End of last)，赋予给中文词。

4. **一对多 (One-to-Many)**:
   - **情况**: 英文 "Hello" (1个词, 0.5秒) -> 中文 "你好" (2个字)。
   - **规则**: 将源单词的 0.5秒时长平均分配给 "你" 和 "好"。

5. **虚词/省略 (Omission)**:
   - **情况**: 英文 "This is a cat" 中的 "a" 在中文中通常不翻译。
   - **规则**: 将未翻译的虚词（如 a, the, of）的时间戳，合并到它紧邻的、有实际意义的名词或动词上。

# IO Format

## Input Format
JSON 数组，包含 `id`, `text`, `words` (words 包含 `word`, `start`, `end`)。

## Output Format
返回纯 JSON 格式（不要用 Markdown 代码块包裹），数组结构如下：
- `id`: 保持不变
- `src_text`: 原文
- `trans_text`: 译文
- `src_words`: 原始单词列表（保持不变）
- `trans_words`: 新生成的中文列表。**注意**：`trans_words` 里的对象必须包含 `word`, `start`, `end`。

# Examples (Few-Shot Learning)

## Example 1: 简单句 + 一对多
**Input:**
[
  {
    "id": 1,
    "text": "Hello world",
    "words": [
      {"word": " Hello", "start": 0.5, "end": 0.8},
      {"word": " world", "start": 0.8, "end": 1.5}
    ]
  }
]
**Output:**
[
  {
    "id": 1,
    "src_text": "Hello world",
    "trans_text": "你好世界",
    "src_words": [
      {"word": " Hello", "start": 0.5, "end": 0.8},
      {"word": " world", "start": 0.8, "end": 1.5}
    ],
    "trans_words": [
      {"word": "你好", "start": 0.5, "end": 0.8},
      {"word": "世界", "start": 0.8, "end": 1.5}
    ]
  }
]

## Example 2: 语序颠倒 (倒装句)
**Input:**
[
  {
    "id": 2,
    "text": "The apple that I ate",
    "words": [
      {"word": " The", "start": 1.0, "end": 1.2},
      {"word": " apple", "start": 1.2, "end": 1.8},
      {"word": " that", "start": 1.8, "end": 2.0},
      {"word": " I", "start": 2.0, "end": 2.2},
      {"word": " ate", "start": 2.2, "end": 2.8}
    ]
  }
]
**Output:**
[
  {
    "id": 2,
    "src_text": "The apple that I ate",
    "trans_text": "我吃的那个苹果",
    "src_words": [ ... ],
    "trans_words": [
      {"word": "我", "start": 2.0, "end": 2.2},    // 对应 I
      {"word": "吃的", "start": 2.2, "end": 2.8},  // 对应 ate
      {"word": "那个", "start": 1.0, "end": 1.2},  // 对应 The (合并 that/The的时间或调整)
      {"word": "苹果", "start": 1.2, "end": 1.8}   // 对应 apple
    ]
  }
]

## Example 3: 虚词省略与多对一
**Input:**
[
  {
    "id": 3,
    "text": "It is a cat",
    "words": [
      {"word": " It", "start": 3.0, "end": 3.2},
      {"word": " is", "start": 3.2, "end": 3.4},
      {"word": " a", "start": 3.4, "end": 3.5},
      {"word": " cat", "start": 3.5, "end": 4.0}
    ]
  }
]
**Output:**
[
  {
    "id": 3,
    "src_text": "It is a cat",
    "trans_text": "它是一只猫",
    "src_words": [ ... ],
    "trans_words": [
      {"word": "它", "start": 3.0, "end": 3.2},
      {"word": "是", "start": 3.2, "end": 3.4},
      {"word": "一只", "start": 3.4, "end": 3.5}, // "a" 的时间很短，这里可以稍微借用 "cat" 的前段，或者直接对应 "a"
      {"word": "猫", "start": 3.5, "end": 4.0}
    ]
  }
]

# Task
现在，请严格按照上述规则和格式，处理以下数据：
```

### Structured outputs

```
{
  "type": "object",
  "properties": {
    "segments": {
      "type": "array",
      "description": "The list of aligned subtitle segments.",
      "items": {
        "type": "object",
        "properties": {
          "id": {
            "type": "integer"
          },
          "src_text": {
            "type": "string"
          },
          "trans_text": {
            "type": "string"
          },
          "src_words": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "word": {
                  "type": "string"
                },
                "start": {
                  "type": "number"
                },
                "end": {
                  "type": "number"
                }
              },
              "required": [
                "word",
                "start",
                "end"
              ],
              "propertyOrdering": [
                "word",
                "start",
                "end"
              ]
            }
          },
          "trans_words": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "word": {
                  "type": "string"
                },
                "start": {
                  "type": "number"
                },
                "end": {
                  "type": "number"
                }
              },
              "required": [
                "word",
                "start",
                "end"
              ],
              "propertyOrdering": [
                "word",
                "start",
                "end"
              ]
            }
          }
        },
        "required": [
          "id",
          "src_text",
          "trans_text",
          "src_words",
          "trans_words"
        ],
        "propertyOrdering": [
          "id",
          "src_text",
          "trans_text",
          "src_words",
          "trans_words"
        ]
      }
    }
  },
  "required": [
    "segments"
  ],
  "propertyOrdering": [
    "segments"
  ]
}
```

### 💡 使用技巧

1.  **JSON 验证**: AI 生成结果后，建议找一个 JSON 校验工具（或直接用代码 `json.loads`）验证一下格式是否正确。
2.  **Example 2 的说明**: 在“语序颠倒”的例子中，我特意在 Output 里展示了**非线性时间轴**（即：2.0s 的词排在 1.2s 的词前面）。
    *   如果前端播放器是基于 `currentTime` 来高亮的，这没问题（播放到 1.2s 时“苹果”亮，播放到 2.0s 时“我”亮）。
    *   如果这会让你的前端逻辑崩溃，请告诉我，我可以修改 Prompt 让 AI 强制进行“线性平滑处理”（即牺牲精准度，保证时间递增）。

# 阶段三 将 JSON 转换为 ASS 字幕

## 第一步 上传文件

将 AI 处理后的 Json 文件上传

In [ ]:
import os
import json
from google.colab import files

print("📂 请点击下方按钮上传文件...")
uploaded = files.upload()

if not uploaded:
    print("❌ 未上传文件，请重新运行。")
else:
    # 获取文件名
    input_json_filename = next(iter(uploaded))
    print(f"✅ 文件已接收: {input_json_filename}")

    # 读取数据以供下一步使用
    try:
        with open(input_json_filename, 'r', encoding='utf-8') as f:
            subtitle_data = json.load(f)
        print("📊 JSON 数据读取成功，准备处理...")
    except Exception as e:
        print(f"❌ JSON 读取失败: {e}")

## 第二步 处理并保存

In [ ]:
import math
import os

def format_time_srt(seconds):
    """秒转 SRT 时间格式 HH:MM:SS,mmm"""
    millis = int((seconds - int(seconds)) * 1000)
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{secs:02d},{millis:03d}"

def format_time_ass(seconds):
    """秒转 ASS 时间格式 H:MM:SS.cc"""
    centis = int((seconds - int(seconds)) * 100)
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours}:{minutes:02d}:{secs:02d}.{centis:02d}"

def generate_karaoke_text(words_list):
    """生成 ASS 卡拉OK 文本"""
    if not words_list: return ""
    line_text = ""
    # 简单的容错：如果没有 start，默认取上一词的结束或0
    current_cursor = words_list[0].get('start', 0)

    for w in words_list:
        start = w.get('start', current_cursor)
        end = w.get('end', start)
        text = w.get('word', "")

        # 计算空隙 (Gap)
        gap = start - current_cursor
        if gap > 0.01:
            line_text += f"{{\\k{int(gap*100)}}}"

        # 计算单词持续时间
        duration = end - start
        k_val = int(duration * 100)
        if k_val < 1: k_val = 1

        line_text += f"{{\\k{k_val}}}{text}"
        current_cursor = end
    return line_text

def create_ass_file(segments, is_source=True, filename="output.ass"):
    """生成单语 ASS 文件"""
    header = """[Script Info]
ScriptType: v4.00+
PlayResX: 1920
PlayResY: 1080
[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Default,Arial,60,&H00FFFFFF,&H0000FFFF,&H00000000,&H80000000,-1,0,0,0,100,100,0,0,1,2.0,1.0,2,10,10,30,1
[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""
    events = []
    text_key = 'src_words' if is_source else 'trans_words'

    for seg in segments:
        words = seg.get(text_key, [])
        if not words: continue

        start_t = format_time_ass(words[0]['start'])
        end_t = format_time_ass(words[-1]['end'])
        k_text = generate_karaoke_text(words)

        events.append(f"Dialogue: 0,{start_t},{end_t},Default,,0,0,0,,{k_text}")

    with open(filename, 'w', encoding='utf-8') as f:
        f.write(header + "\n".join(events))

def create_bilingual_ass_file(segments, filename="output_bilingual.ass"):
    """生成双语 ASS 文件 (主字幕在下，副字幕在上)"""
    # 定义两种样式：Main(大字,底部), Sub(小字,上方)
    header = """[Script Info]
ScriptType: v4.00+
PlayResX: 1920
PlayResY: 1080
[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Main,Arial,64,&H00FFFFFF,&H0000FFFF,&H00000000,&H80000000,-1,0,0,0,100,100,0,0,1,2.0,1.0,2,10,10,20,1
Style: Sub,Arial,40,&H00FFFFFF,&H00AAAAAA,&H00000000,&H80000000,0,0,0,0,100,100,0,0,1,1.5,1.0,2,10,10,95,1
[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""
    events = []

    for seg in segments:
        # 获取源语言和目标语言的单词列表
        src_w = seg.get('src_words', [])
        trans_w = seg.get('trans_words', [])

        if not src_w and not trans_w: continue

        # 1. 添加翻译层 (Main Style - 底部)
        if trans_w:
            start_t = format_time_ass(trans_w[0]['start'])
            end_t = format_time_ass(trans_w[-1]['end'])
            k_text = generate_karaoke_text(trans_w)
            events.append(f"Dialogue: 0,{start_t},{end_t},Main,,0,0,0,,{k_text}")

        # 2. 添加原文层 (Sub Style - 顶部，稍微淡一点的填充色)
        if src_w:
            start_t = format_time_ass(src_w[0]['start'])
            end_t = format_time_ass(src_w[-1]['end'])
            k_text = generate_karaoke_text(src_w)
            events.append(f"Dialogue: 0,{start_t},{end_t},Sub,,0,0,0,,{k_text}")

    with open(filename, 'w', encoding='utf-8') as f:
        f.write(header + "\n".join(events))

def create_srt_file(segments, is_source=True, filename="output.srt"):
    """生成 SRT 文件内容"""
    text_key = 'src_text' if is_source else 'trans_text'
    # 备用：从单词拼凑
    word_key = 'src_words' if is_source else 'trans_words'

    with open(filename, 'w', encoding='utf-8') as f:
        for idx, seg in enumerate(segments):
            words = seg.get(word_key, [])
            if not words: continue

            start_t = format_time_srt(words[0]['start'])
            end_t = format_time_srt(words[-1]['end'])

            text = seg.get(text_key, "")
            if not text:
                text = "".join([w['word'] for w in words])

            f.write(f"{idx+1}\n{start_t} --> {end_t}\n{text}\n\n")

# --- 主执行逻辑 ---
# 确保 subtitle_data 存在 (来自第一步)
if 'subtitle_data' not in globals():
    print("❌ 错误：请先运行第一步上传文件！")
else:
    base_name = os.path.splitext(input_json_filename)[0]
    segments = subtitle_data.get('segments', [])
    if isinstance(subtitle_data, list): segments = subtitle_data

    # 定义文件名 (5个文件)
    files_to_generate = {
        'src_ass': f"{base_name}_source.ass",
        'trans_ass': f"{base_name}_translated.ass",
        'bilingual_ass': f"{base_name}_bilingual.ass", # 新增
        'src_srt': f"{base_name}_source.srt",
        'trans_srt': f"{base_name}_translated.srt"
    }

    print(f"⚙️ 正在生成 5 个字幕文件 (包含双语ASS)...")

    # 生成所有文件
    create_ass_file(segments, is_source=True, filename=files_to_generate['src_ass'])
    create_ass_file(segments, is_source=False, filename=files_to_generate['trans_ass'])
    create_bilingual_ass_file(segments, filename=files_to_generate['bilingual_ass']) # 新增调用
    create_srt_file(segments, is_source=True, filename=files_to_generate['src_srt'])
    create_srt_file(segments, is_source=False, filename=files_to_generate['trans_srt'])

    print("✅ 生成完毕，文件列表：")
    for key, val in files_to_generate.items():
        print(f"  - {val}")

## 第三步 下载处理后的文件

In [ ]:
import zipfile
from google.colab import files

zip_filename = f"{base_name}_all_subtitles.zip"

print(f"📦 正在打包文件为 {zip_filename} ...")

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # 遍历上一步生成的文件字典并写入压缩包
    for key, fname in files_to_generate.items():
        if os.path.exists(fname):
            zipf.write(fname)
            print(f"  -> 已添加: {fname}")
        else:
            print(f"  ⚠️ 警告: 找不到文件 {fname}")

print("-" * 30)
print("🚀 准备下载...")

try:
    files.download(zip_filename)
except Exception as e:
    print(f"❌ 自动下载失败: {e}")
    print("👉 请在左侧【文件】侧边栏手动右键下载 .zip 文件。")

## 第四步 烧录字幕文件到视频（测试）

> 生成黑屏测试视频 (烧录字幕)

In [ ]:
# @markdown **🛠️ 前置修复：安装中文字体**
# @markdown 运行此格以解决字幕方块问题。运行一次即可。

!sudo apt-get update -y
!sudo apt-get install -y fonts-noto-cjk extra-runtime-dependencies
!fc-cache -fv

print("-" * 30)
print("✅ 中文字体 (Noto Sans CJK SC) 安装完成！")

In [ ]:
import math
import os

def format_time_srt(seconds):
    """秒转 SRT 时间格式 HH:MM:SS,mmm"""
    millis = int((seconds - int(seconds)) * 1000)
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{secs:02d},{millis:03d}"

def format_time_ass(seconds):
    """秒转 ASS 时间格式 H:MM:SS.cc"""
    centis = int((seconds - int(seconds)) * 100)
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours}:{minutes:02d}:{secs:02d}.{centis:02d}"

def generate_karaoke_text(words_list):
    """生成 ASS 卡拉OK 文本"""
    if not words_list: return ""
    line_text = ""
    current_cursor = words_list[0].get('start', 0)

    for w in words_list:
        start = w.get('start', current_cursor)
        end = w.get('end', start)
        text = w.get('word', "")

        # 计算空隙
        gap = start - current_cursor
        if gap > 0.01:
            line_text += f"{{\\k{int(gap*100)}}}"

        # 计算单词持续时间
        duration = end - start
        k_val = int(duration * 100)
        if k_val < 1: k_val = 1

        line_text += f"{{\\k{k_val}}}{text}"
        current_cursor = end
    return line_text

def create_ass_file(segments, is_source=True, filename="output.ass"):
    """生成单语 ASS 文件"""
    font_name = "Noto Sans CJK SC"

    header = f"""[Script Info]
ScriptType: v4.00+
PlayResX: 1920
PlayResY: 1080
[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Default,{font_name},60,&H00FFFFFF,&H0000FFFF,&H00000000,&H80000000,-1,0,0,0,100,100,0,0,1,2.0,1.0,2,10,10,30,1
[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""
    events = []
    text_key = 'src_words' if is_source else 'trans_words'

    for seg in segments:
        words = seg.get(text_key, [])
        if not words: continue

        start_t = format_time_ass(words[0]['start'])
        end_t = format_time_ass(words[-1]['end'])
        k_text = generate_karaoke_text(words)

        events.append(f"Dialogue: 0,{start_t},{end_t},Default,,0,0,0,,{k_text}")

    with open(filename, 'w', encoding='utf-8') as f:
        f.write(header + "\n".join(events))

def create_bilingual_ass_file(segments, filename="output_bilingual.ass"):
    """生成双语 ASS 文件"""
    # 如果你想英文用 Arial，中文用 Noto，可以分开写，但为了保险起见，这里统一切换为 Noto (支持中英)
    font_name = "Noto Sans CJK SC"

    header = f"""[Script Info]
ScriptType: v4.00+
PlayResX: 1920
PlayResY: 1080
[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Main,{font_name},64,&H00FFFFFF,&H0000FFFF,&H00000000,&H80000000,-1,0,0,0,100,100,0,0,1,2.0,1.0,2,10,10,20,1
Style: Sub,{font_name},40,&H00FFFFFF,&H00AAAAAA,&H00000000,&H80000000,0,0,0,0,100,100,0,0,1,1.5,1.0,2,10,10,95,1
[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""
    events = []

    for seg in segments:
        src_w = seg.get('src_words', [])
        trans_w = seg.get('trans_words', [])

        if not src_w and not trans_w: continue

        if trans_w:
            start_t = format_time_ass(trans_w[0]['start'])
            end_t = format_time_ass(trans_w[-1]['end'])
            k_text = generate_karaoke_text(trans_w)
            events.append(f"Dialogue: 0,{start_t},{end_t},Main,,0,0,0,,{k_text}")

        if src_w:
            start_t = format_time_ass(src_w[0]['start'])
            end_t = format_time_ass(src_w[-1]['end'])
            k_text = generate_karaoke_text(src_w)
            events.append(f"Dialogue: 0,{start_t},{end_t},Sub,,0,0,0,,{k_text}")

    with open(filename, 'w', encoding='utf-8') as f:
        f.write(header + "\n".join(events))

def create_srt_file(segments, is_source=True, filename="output.srt"):
    """生成 SRT 文件"""
    text_key = 'src_text' if is_source else 'trans_text'
    word_key = 'src_words' if is_source else 'trans_words'

    with open(filename, 'w', encoding='utf-8') as f:
        for idx, seg in enumerate(segments):
            words = seg.get(word_key, [])
            if not words: continue

            start_t = format_time_srt(words[0]['start'])
            end_t = format_time_srt(words[-1]['end'])

            text = seg.get(text_key, "")
            if not text:
                text = "".join([w['word'] for w in words])

            f.write(f"{idx+1}\n{start_t} --> {end_t}\n{text}\n\n")

# --- 主执行逻辑 ---
if 'subtitle_data' not in globals():
    print("❌ 错误：请先运行第一步上传文件！")
else:
    base_name = os.path.splitext(input_json_filename)[0]
    segments = subtitle_data.get('segments', [])
    if isinstance(subtitle_data, list): segments = subtitle_data

    files_to_generate = {
        'src_ass': f"{base_name}_source.ass",
        'trans_ass': f"{base_name}_translated.ass",
        'bilingual_ass': f"{base_name}_bilingual.ass",
        'src_srt': f"{base_name}_source.srt",
        'trans_srt': f"{base_name}_translated.srt"
    }

    print(f"⚙️ 正在生成字幕文件 (字体已修正为 Noto Sans CJK SC)...")

    create_ass_file(segments, is_source=True, filename=files_to_generate['src_ass'])
    create_ass_file(segments, is_source=False, filename=files_to_generate['trans_ass'])
    create_bilingual_ass_file(segments, filename=files_to_generate['bilingual_ass'])
    create_srt_file(segments, is_source=True, filename=files_to_generate['src_srt'])
    create_srt_file(segments, is_source=False, filename=files_to_generate['trans_srt'])

    print("✅ 字幕文件更新完毕。请继续运行【第四步】进行测试。")

In [ ]:
import os
import re
import subprocess
import shutil
from google.colab import files
from tqdm.notebook import tqdm

# @markdown **1. 视频配置**
video_width = 1920 # @param {type:"integer"}
video_height = 1080 # @param {type:"integer"}
video_fps = 30 # @param {type:"integer"}

# @markdown **2. 时间范围配置** (单位：秒)
# @markdown * `start_time`: 视频开始时间点（对应字幕时间轴）。
# @markdown * `end_time`: 视频结束时间点。**设为 0 则自动分析字幕结尾。**
start_time = 600 # @param {type:"number"}
end_time = 1100 # @param {type:"number"}

# @markdown **3. 字幕文件** (留空自动寻找双语字幕)
subtitle_file_path = "" # @param {type:"string"}

# @markdown **4. 输出文件名**
output_test_filename = "subtitle_test_preview.mp4" # @param {type:"string"}

# --- 自动寻找字幕文件 ---
if not subtitle_file_path:
    candidates = [f for f in os.listdir('.') if f.endswith('_bilingual.ass')]
    if candidates:
        subtitle_file_path = os.path.abspath(candidates[0])
    else:
        candidates_all = [f for f in os.listdir('.') if f.endswith('.ass')]
        if candidates_all:
            subtitle_file_path = os.path.abspath(candidates_all[0])

def get_ass_max_time(file_path):
    """从 ASS 文件中解析最后一条字幕的结束时间"""
    max_seconds = 0
    time_pattern = re.compile(r'(\d+):(\d{2}):(\d{2})\.(\d{2})')
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.startswith('Dialogue:'):
                    parts = line.split(',')
                    if len(parts) > 2:
                        end_time_str = parts[2].strip()
                        match = time_pattern.match(end_time_str)
                        if match:
                            h, m, s, cs = map(int, match.groups())
                            total_sec = h * 3600 + m * 60 + s + cs / 100.0
                            if total_sec > max_seconds:
                                max_seconds = total_sec
    except Exception as e:
        print(f"⚠️ 警告: 字幕解析出错 ({e})，将使用默认时长。")
        return 10
    return max_seconds

def parse_ffmpeg_time_str(time_str):
    """将 FFmpeg 日志中的 time=HH:MM:SS.ms 转换为秒数"""
    try:
        h, m, s = time_str.split(':')
        return int(h) * 3600 + int(m) * 60 + float(s)
    except:
        return 0

def burn_video_segment():
    # 1. 检查文件
    if not subtitle_file_path or not os.path.exists(subtitle_file_path):
        print(f"❌ 错误：找不到字幕文件。请检查上一步是否成功生成，或手动填入路径。")
        return

    print(f"ℹ️ 选中字幕文件: {os.path.basename(subtitle_file_path)}")

    # 2. 计算时间范围
    print(f"⏱️ 正在计算时间范围...")
    detected_max_time = get_ass_max_time(subtitle_file_path)

    final_start = max(0, start_time)
    final_end = end_time

    if final_end <= 0 or final_end <= final_start:
        final_end = detected_max_time + 2 # 自动结尾 + 缓冲
        print(f"   -> 自动检测字幕结束时间: {detected_max_time:.2f}s")

    duration = final_end - final_start

    print("-" * 30)
    print(f"🚀 开始生成预览视频 (FFmpeg)")
    print(f"   -> 时间区间: {final_start}s 至 {final_end}s (总长: {duration:.2f}s)")
    print(f"   -> 分辨率: {video_width}x{video_height}, FPS: {video_fps}")
    print("-" * 30)

    # 3. 准备 FFmpeg 命令
    # 复制字幕文件，避免路径转义问题
    safe_sub_path = "temp_burn.ass"
    shutil.copy(subtitle_file_path, safe_sub_path)

    # 核心逻辑：
    # -f lavfi -i color : 生成无限长的黑屏流
    # -ss : 指定开始时间 (Output option)
    # -to : 指定结束时间 (Output option)
    # 注意：lavfi 本身没有时间戳概念，但配合 -vf ass，字幕会根据绝对时间渲染。
    # 我们让 input 持续产生，然后用 -ss 和 -to 截取输出段。

    cmd = [
        'ffmpeg', '-y',
        '-f', 'lavfi',
        '-i', f'color=c=black:s={video_width}x{video_height}:r={video_fps}',
        '-ss', str(final_start), # 从这里开始截取
        '-to', str(final_end),   # 到这里结束
        '-vf', f"ass={safe_sub_path}",
        '-c:v', 'libx264',
        '-preset', 'ultrafast',  # 预览追求极速
        '-pix_fmt', 'yuv420p',
        output_test_filename
    ]

    # 4. 执行并监控进度
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    )

    # 进度条 (总长度为 duration)
    pbar = tqdm(total=duration, unit="s", bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt}s [{elapsed}<{remaining}]")

    # 匹配 time=00:00:00.00
    time_p = re.compile(r'time=(\d{2}:\d{2}:\d{2}\.\d+)')
    last_processed_time = 0

    try:
        # 读取 stderr
        for line in process.stderr:
            match = time_p.search(line)
            if match:
                current_time_str = match.group(1)
                # FFmpeg 这里的 time 是相对于输出文件的当前时长 (从 0 开始计数)
                current_processed_time = parse_ffmpeg_time_str(current_time_str)

                increment = current_processed_time - last_processed_time
                if increment > 0:
                    pbar.update(increment)
                    last_processed_time = current_processed_time
    except Exception as e:
        print(f"⚠️ 进度显示异常: {e}")

    process.wait()
    pbar.close()

    if process.returncode == 0:
        file_size_mb = os.path.getsize(output_test_filename) / (1024 * 1024)
        print(f"✅ 视频生成成功: {output_test_filename} ({file_size_mb:.2f} MB)")

        if os.path.exists(safe_sub_path):
            os.remove(safe_sub_path)

        try:
            files.download(output_test_filename)
        except:
            print("⚠️ 自动下载被拦截，请手动下载。")
    else:
        print("❌ 视频生成失败！建议检查时间范围是否超出字幕长度。")

# 执行
burn_video_segment()